In [5]:
import numpy as np
import os
from tqdm import tqdm_notebook
import scipy.io as scio

In [22]:
########################################################################################################################
# find peak
def find_peak(x):
    peak = []
    for index in range(1, len(x)-1, 1):
        if x[index-1] < x[index] and x[index] > x[index+1]:
            peak.append(x[index])
    return peak


########################################################################################################################
# 背景扣除
def simulate_background(x):
    x = x.tolist()
    peak_list = find_peak(x)
    peak_position_list = [i for i, x in enumerate(x) if x in peak_list]
    min_list = [min(x[peak_position_list[i]:peak_position_list[i+1]]) for i in range(len(peak_position_list)-1)]
    min_new_list = min_list
    for index in range(1, len(min_list)-1, 1):
        if min_list[index] != 0:
            if min_list[index-1] != 0 and min_list[index+1] != 0:
                #if 1.2*min_list[index+1] > min_list[index] > 1.2*min_list[index-1]:
                #    min_list[index] = np.min([min_list[index-1], min_list[index+1]])
                #if 1.2*min_list[index+1] < min_list[index] < 1.2*min_list[index-1]:
                #    min_list[index] = np.min([min_list[index-1], min_list[index+1]])
                if min_list[index] > 1.2*min_list[index-1] and min_list[index] > 1.2*min_list[index+1]:
                    min_new_list[index] = 0.5*(min_list[index-1] + min_list[index+1])
    for index in range(len(peak_position_list)-1):
        x_change = x[peak_position_list[index]:peak_position_list[index+1]]
        x_change = [min_new_list[index] for i in range(len(x_change))]
        x[peak_position_list[index]:peak_position_list[index + 1]] = x_change
    x = np.array(x)
    return x


########################################################################################################################
# 背景扣除
def correct_background(x):
    x = np.maximum(x, 0)
    x = x-simulate_background(x)
    return x

In [ ]:

point_number = 3
orig_path = r'D:\data\china_2020\pixel_select_dark_corrected'
rock_type = ['S2', 'S5', 'S17', 'S19', 'S30']
process_path = ['DenoisedSpectrum', 'SmoothedSpectrum', 'RawSpectrum']
save_path_root = r'D:\data\china_2020\pixel_select_background_corrected'
pixel_list = np.loadtxt(r'D:\data\china_2020\pixel_list.txt')
for state in tqdm_notebook(['rock', 'pellet']):
    for i in tqdm_notebook(rock_type):
        for j in process_path:
            for z in range(point_number):
                data = np.loadtxt(os.path.join(orig_path, i + '-' + state + '_' + j + '_' + str(z + 1) + '.txt'))
                data = data[:, 1:]
                data_res = np.zeros((data.shape))
                for index_i in range(data.shape[1]):
                    data_res[:, index_i] = correct_background(data[:, index_i])
                np.savetxt(os.path.join(save_path_root, i + '-' + state + '_' + j + '_' + str(z + 1) + '.txt'), np.column_stack((pixel_list, data_res)), '%.5g')
